In [ ]:
#@title 🚀 Yannova 3D Generator (IMAGE + TEXT) 
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers accelerate trimesh flask flask-cors pyngrok
!pip install -q git+https://github.com/openai/shap-e.git

import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import trimesh, base64, io
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"GPU: {torch.cuda.is_available()}")

print("Loading Transmitter...")
xm = load_model('transmitter', device=device)

print("Loading Text Model...")
text_model = load_model('text300M', device=device)

print("Loading Image Model...")
image_model = load_model('image300M', device=device)

diffusion = diffusion_from_config(load_config('diffusion'))

app = Flask(__name__)
CORS(app)

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data.get('prompt', 'a door')
    image_data = data.get('image', None)
    
    print(f"Request recieved. Prompt: {prompt}, Image: {'Yes' if image_data else 'No'}")

    try:
        if image_data:
            # Image-to-3D Generation
            print("Genereren van afbeelding...")
            image_bytes = base64.b64decode(image_data)
            img = Image.open(io.BytesIO(image_bytes)).convert('RGB')
            img = img.resize((256, 256)) # Normalize size

            latents = sample_latents(
                batch_size=1,
                model=image_model,
                diffusion=diffusion,
                guidance_scale=3.0,
                model_kwargs=dict(images=[img]),
                progress=True,
                clip_denoised=True,
                use_fp16=True,
                use_karras=True,
                karras_steps=64,
                sigma_min=1e-3,
                sigma_max=160,
                s_churn=0
            )
        else:
            # Text-to-3D Generation
            print("Genereren van tekst...")
            latents = sample_latents(
                batch_size=1,
                model=text_model,
                diffusion=diffusion,
                guidance_scale=15.0,
                model_kwargs=dict(texts=[prompt]),
                progress=True,
                clip_denoised=True,
                use_fp16=True,
                use_karras=True,
                karras_steps=64,
                sigma_min=1e-3,
                sigma_max=160,
                s_churn=0
            )

        mesh = decode_latent_mesh(xm, latents[0]).tri_mesh()
        tri = trimesh.Trimesh(vertices=mesh.verts, faces=mesh.faces)
        return jsonify({"success": True, "model": base64.b64encode(tri.export(file_type='glb')).decode()})
    except Exception as e:
        print(f"FOUT: {e}")
        return jsonify({"success": False, "error": str(e)}), 500

@app.route('/health')
def health():
    return jsonify({"status": "ok"})

# Jouw token wordt automatisch gebruikt indien eerder gezet, anders vervang 'TOKEN' hieronder.
ngrok.set_auth_token("2sWgeBayoMfOErTfRRjj84tEIiy_5qSmDeZRgBfdw86VChh4i")
try:
    url = ngrok.connect(5000)
    print(f"\n🌐 NIEUWE URL: {url}\n")
except Exception as e:
    print("Kon geen verbinding maken, misschien draait er al een tunnel?", e)

app.run(port=5000)